In [ ]:
import sys
import os

prob_model_dir = "/home/lomereiter/github/probModel"
sys.path.append(prob_model_dir)
sys.path.append(os.path.dirname(prob_model_dir))
os.chdir(prob_model_dir)

%load_ext autoreload
%autoreload
from probModel.pipeline2 import ProbPipeline
import json
config = json.loads(open(prob_model_dir + "/decoy.json").read())
pipeline = ProbPipeline(config)
pipeline.initialize()

names = ['+'.join([f,a]) for f in pipeline.sum_formulae for a in pipeline.adducts]

# read expected coefficients
from collections import defaultdict
expected = defaultdict(float)
total = 0.0
for line in open(prob_model_dir + "/hmdb_sim_list.txt").readlines()[1:]:
    f, a, m = line.split()
    f=f[1:-1]
    a=a[1:-1]
    expected[f+'+'+a] = float(m)
    total += float(m)

In [ ]:
%autoreload
# if rho is not provided, it's computed conservatively to ensure convergence, but the convergence is very slow
# too low value for rho can lead to funny pictures having nothing to do with reality
solver = pipeline.get_solver(theta=0.5, lambda_=1.0, rho=5.0)


In [ ]:
%autoreload
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from IPython import display
for i in range(1000):
    solver.run_single_iteration(safe_step_size=False)
    # solver.run_single_iteration(safe_step_size=True)
    # run_single_iteration has a parameter safe_step_size which enables line search to ensure convergence 
    # (there is no scientific proof that it works, but practice suggests that it does)
    # with this param set to true we get convergence for any value of rho, but it requires ~2x more computations
    if i > 0 and i % 5 == 0:
        #plt.subplot(211)
        plt.imshow(solver.get_image(solver.w_hat, 0))
        data = []
        wrong = 0.0
        abundancies = solver.w_hat.sum(axis=1)
        for f, a in zip(names, abundancies/abundancies.sum() * total):
            if expected[f] > 0:
                data.append((a, expected[f]))
            elif a > 0:
                wrong += a
        xs = [y for x, y in data]
        ys = [x for x, y in data]
        #plt.subplot(212)
        #plt.scatter(xs, ys)
        # correlation coefficient between true and estimated coefficients
     
        display.clear_output(wait=True)
        display.display(plt.gcf())
        print "correlation of coefficients:", np.corrcoef(xs, ys)[0,1]
        print "proportion of wrongly assigned abundancies:", wrong/total # the percentage of molecule abundancies assigned to non-existent molecules
        print "total noise intensity:", solver.w_hat[-1,:].sum()
        print "iterations performed:", i
        #print np.linalg.norm(solver.G.T.dot(solver.w_hat.T)) / np.linalg.norm(solver.w_hat)
        print solver.LL(solver.w_hat)